<a href="https://colab.research.google.com/github/immohann/ms-apriori/blob/main/MS_Aprioi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import itertools
from collections import defaultdict, OrderedDict

# custom 
def readInput(inputfile):
    "read inputfile"
    data = []
    with open(inputfile) as myfile:
      inp = []
      csv_records = csv.reader(myfile)
      for row in csv_records:
        new_list = [int(item) for item in row]
        data.append(new_list)
    return data

# custom 
def readParam(paramFile):
    "Read the parameter file"

    param = {}
    sdc = 0

    data = readInput("input-data.txt")
    uniq_vals = set(itertools.chain.from_iterable(data))

    file_iter = open("parameter-file.txt", 'r')
    for line in file_iter:
        line = line.replace(" ", "")
        line = line.strip()
        if 'MIS' in line:
            key = line[line.find("(") + 1:line.find(")")]
            val = line[line.find("=") + 1:]
            param[key] = float(val)
            
        elif 'SDC' in line:
            sdc = float(line[line.find("=") + 1:])

    # set rest value to te rest of the elements
    rest = []
    for key,vals in param.items():
      if key == "rest":
        r = vals
      else:
        rest.append(int(key))

    for i in uniq_vals:
      if i not in rest:
        param[str(i)] = r
    del param['rest']

    param = {int(k):float(v) for k,v in param.items()}
    param = OrderedDict(sorted(param.items()))

    return param, sdc

# custom x modified
def getItemSetTransactionList(data_iterator):
    "returns transactionList, 1-itemSet"

    transactionList = []
    itemSet = set()
    for record in data_iterator:
        transaction = set(record)
        transactionList.append(transaction)
        for item in transaction:
            itemSet.add(item)              
            # Generate 1-itemSets
    return itemSet, transactionList

# custom x modified
def joinSet(itemSet, length):
    """Join a set with itself and returns the n-element itemsets"""
    return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])

# to-be modified
def level2_cand_gen(FF, param, sdc, count, maxMIS, transSize):
    "generate cadidates for the 2-itemset"
    s = set(frozenset([w]) for w in FF)
    s2 = joinSet(s, 2)

    Cand = []
    for e in s2:
        minSup = 1
        maxSup = 0

        for r in e:
            if count[r]/float(transSize) < minSup:
                minSup = count[r]/float(transSize)
            if count[r]/float(transSize) > maxSup:
                maxSup = count[r]/float(transSize)

        if maxSup-minSup <= sdc:
            Cand.append(e)

    ## get the lowest MIS (done)
    minMIS = dict()

    for i in Cand:
        min = 1
        max = 0
        maxMISItem = 0
        for j in i:
            if param[j] < min:
                min = param[j]
            if param[j] > max:
                max = param[j]
                maxMISItem = count[j]
        minMIS[i] = min
        maxMIS[i] = maxMISItem
    return Cand, minMIS

# to-be modified
def level_k_cand_gen(L, param, sdc, k, count, maxMIS):
    s2 = joinSet(L, k)

    Cand = []
    for e in s2:
        minSup = 1
        maxSup = 0
        for r in e:
            if param[r] < minSup:
                minSup = param[r]
            if param[r] > maxSup:
                maxSup = param[r]
        if maxSup-minSup <= sdc:
            Cand.append(e)


    ## get the lowest MIS
    minMIS = dict()
    for i in Cand:
        min = 1
        max = 0
        maxMISItem = 0
        for j in i:
            # print(j)
            if param[j] < min:
                min = param[j]
            if param[j] > max:
                max = param[j]
                maxMISItem = count[j]
        minMIS[i] = min
        maxMIS[i] = maxMISItem
    
    return Cand, minMIS


def writeOutputFile(outputFile, F, count, maxMIS):
    "Save output in the same format as requested"
    with open(outputFile, 'w') as f:
        k = []
        for key in F:
            k.append(key)
        k = sorted(k)
        for i in k:
            f.write("Frequent %d-itemsets\n\n" % i)
            for j in F[i]:
                if i == 1:
                    tmp = next(iter(j))
                    f.write("\t%d : %s \n" % (count[tmp], j))
                else:
                    f.write("\t%d : %s \n" % (count[j], set(j)))
                    if i != 1:
                        # TODO: extract the correct tail
                        #end = "10"
                        f.write("Tailcount = %d \n" % maxMIS[j])

            f.write("\n\tTotal number of frequent %d-itemsets = %d \n" % (i, len(F[i])))
            f.write("\n\n")
            
# to-be modified
def MSA (fileData, parameters, sdc):
    "MSApriori algorithm"
    one, trans = getItemSetTransactionList(fileData)
    sorted_items = sorted(parameters.items())
    print(sorted_items)
    # sorted_items = sorted(parameters.items(), key=operator.itemgetter(1))
    transSize = len(trans)

    print("Working on 1 itemset ......")
    ## construct the M
    M =[]
    for i in (sorted_items):
        M.append(i[0])

    ## get the count
    count=defaultdict(int)
    for item in M:
        for transaction in trans:
            m={item}
            if m.issubset(transaction):
                count[item] += 1

    ## Initial pass
    L=[]
    start = 0
    pivot = 0
    for i in M:
        if count[i]/float(len(trans)) >= parameters[i] and start == 0:
            L.append(i)
            start = 1
            pivot = parameters[i]
        elif count[i]/float(len(trans))  >= pivot and start == 1:
            L.append(i)
    print(L)
    ## select the 1-frequentset
    F = dict()
    F_local = []
    for i in L:
        if count[i] / float(len(trans))  >= parameters[i]:
            F_local.append({i})
    ### Check constrains
   
    F[1] = F_local


    ### for the rest of set-size
    k = 2
    maxMIS = dict()
    print("F",F)
    while(len(F[k-1]) != 0):
        print("Working on",k,"itemset ......")
        if(k==2):
            C, minMIS = level2_cand_gen(L, parameters, sdc, count, maxMIS, transSize)
        else:
            C, minMIS = level_k_cand_gen(F[k - 1], parameters, sdc, k, count, maxMIS)

        ## Get the count
        for item in C:
            for transaction in trans:
                if item.issubset(transaction):
                    count[item] += 1

        ## Select the items fro the k-frequent itemset
        F_local = []
        for item in C:
            if count[item] / float(len(trans))  >= minMIS[item]:#parameters[list(item)[0]]:
                F_local.append(item)


        ##### Check constrains
       
        F[k] = F_local

        k+=1
    print("F",F)
    del(F[k-1])
    return F, count, maxMIS

if __name__ == "__main__":

    fileData = readInput("input-data.txt")
    parameters, sdc = readParam("parameter-file.txt")

    F, count, maxMIS = MSA(fileData, parameters, sdc)

    ### Create the outputfile
    writeOutputFile("outFile.txt", F, count, maxMIS)
    print("\n DONE \n")

[(10, 0.43), (20, 0.2), (30, 0.3), (40, 0.4), (50, 0.4), (60, 0.3), (70, 0.2), (80, 0.2), (90, 0.2), (100, 0.1), (120, 0.2), (140, 0.15)]
Working on 1 itemset ......
[20, 30, 50, 70, 80, 90]
F {1: [{20}, {30}, {70}, {80}, {90}]}
Working on 2 itemset ......
Working on 3 itemset ......
F {1: [{20}, {30}, {70}, {80}, {90}], 2: [frozenset({90, 50}), frozenset({80, 20}), frozenset({70, 30})], 3: []}

 DONE 

